In [ ]:
!nvidia-smi

Sun Jun 11 07:59:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

/content
--2023-06-11 08:00:27--  https://docs.google.com/uc?export=download&confirm=&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
Resolving docs.google.com (docs.google.com)... 142.250.125.102, 142.250.125.100, 142.250.125.101, ...
Connecting to docs.google.com (docs.google.com)|142.250.125.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l7rc8mqods1cjrhavidkvl2125hmfogn/1686470400000/04309230031174164349/*/1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-?e=download&uuid=70e67927-8c51-4970-a2d3-74a781b8f719 [following]
--2023-06-11 08:00:28--  https://doc-0s-ag-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l7rc8mqods1cjrhavidkvl2125hmfogn/1686470400000/04309230031174164349/*/1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-?e=download&uuid=70e67927-8c51-4970-a2d3-74a781b8f719
Resolving doc-0s-ag-docs.googleusercontent.com (doc-0s-ag-docs.googleusercontent.com)... 64.2

In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/vehicle-counting.mp4"
SOURCE_VIDEO_PATH

'/content/vehicle-counting.mp4'

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.115  Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete  (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


# Install ByteTrack

In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112
!pip install -q loguru lap

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")




ModuleNotFoundError: ignored

In [ ]:
!pip install thop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)


In [ ]:
import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


# Load pre-trained Model of V8

In [ ]:
MODEL = 'yolov8x.pt'

In [ ]:
from ultralytics import YOLO
model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 139MB/s]
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients


# Predict and annotate single frame

In [ ]:
%cd {HOME}
! yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={SOURCE_VIDEO_PATH}

/content
Ultralytics YOLOv8.0.115 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

video 1/1 (1/538) /content/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 67.3ms
video 1/1 (2/538) /content/vehicle-counting.mp4: 384x640 3 cars, 1 truck, 63.1ms
video 1/1 (3/538) /content/vehicle-counting.mp4: 384x640 4 cars, 2 trucks, 63.1ms
video 1/1 (4/538) /content/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 68.1ms
video 1/1 (5/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 63.1ms
video 1/1 (6/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 63.1ms
video 1/1 (7/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 72.6ms
video 1/1 (8/538) /content/vehicle-counting.mp4: 384x640 5 cars, 1 truck, 63.1ms
video 1/1 (9/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 63.1ms
video 1/1 (10/538) /content/vehicle-counting.mp4: 384x640 4 cars, 1 truck, 64.3ms
video 

In [ ]:
from supervision.draw.color import ColorPalette
from supervision.video.source import get_video_frames_generator
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections,BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator
from supervision.video.sink import VideoInfo,VideoSink
from supervision.geometry.dataclasses import Point
from tqdm.notebook import tqdm
from ultralytics import tracker
from typing import List
import numpy as np

In [ ]:
# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

In [ ]:
OBJECT_NAMES = model.model.names
TARGET_VIDEO_PATH = f"{HOME}/vehicle-counting-result.mp4"
LINE_START = Point(50, 1500)
LINE_END = Point(3840-50, 1500)
CLASS_ID = [2, 3, 5, 7]

In [ ]:
video_info = VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
byte_tracker = BYTETracker(BYTETrackerArgs())

line_counter = LineCounter(start=LINE_START, end=LINE_END)
line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)
box_annotator = BoxAnnotator(color=ColorPalette(),thickness=4,text_thickness=4,text_scale=2.)

with VideoSink(output_path=TARGET_VIDEO_PATH,video_info=video_info) as sink:
  for frame in tqdm(generator, total=video_info.total_frames):
    results = model(frame)[0]



    detections = Detections(
        xyxy= results.boxes.xyxy.cpu().numpy(),
        confidence= results.boxes.conf.cpu().numpy(),
        class_id= results.boxes.cls.cpu().numpy().astype(int),
    )
     # filtering out detections with unwanted classes
    mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
    detections.filter(mask=mask, inplace=True)
    # tracking detections
    tracks = byte_tracker.update(
        output_results=detections2boxes(detections=detections),
        img_info=frame.shape,
        img_size=frame.shape
    )
    tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
    detections.tracker_id = np.array(tracker_id)
    # filtering out detections without trackers
    mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
    detections.filter(mask=mask, inplace=True)
    labels = [
        f'#{tracker_id} {OBJECT_NAMES[class_id]} {conf:0.2f}'
        for _,conf,class_id,tracker_id in detections
    ]

    # updating line counter
    line_counter.update(detections=detections)
    # annotate and display frame
    frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
    line_annotator.annotate(frame=frame, line_counter=line_counter)
    sink.write_frame(frame)

  0%|          | 0/538 [00:00<?, ?it/s]


0: 384x640 3 cars, 1 truck, 63.8ms
Speed: 3.4ms preprocess, 63.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 3 cars, 1 truck, 63.3ms
Speed: 3.9ms preprocess, 63.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 2 trucks, 63.4ms
Speed: 3.8ms preprocess, 63.4ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 1 truck, 63.3ms
Speed: 4.0ms preprocess, 63.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 1 truck, 63.2ms
Speed: 3.2ms preprocess, 63.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 1 truck, 63.5ms
Speed: 3.9ms preprocess, 63.5ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 4 cars, 1 truck, 63.2ms
Speed: 3.2ms preprocess, 63.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 1 truck, 63.1ms
Speed: 4.4ms preprocess, 63.1ms 